In [ ]:
# The following is from "big run"
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

# Setup
lsa_np = np.load('../data/parsed/lsa_output.npy')
metadata = pd.read_pickle('../data/parsed/pickles/pickled_data_test.pickle')
metadata = metadata.loc[metadata['Scenario'] == '401']
metadata = metadata.reset_index(drop=True)
lsa_df = pd.DataFrame(lsa_np)
df = pd.concat([metadata, lsa_df], axis=1, join_axes=[metadata.index])
df = df.loc[df['Label'] != '-1']
df = df.reset_index(drop=True)
cat_features = ['To','From']
features = list(range(100))
features.extend(cat_features + ['Date'])
# features.extend(cat_features + ['ID'])

df = df[features + ['Label'] + ['ID']]
# df = df[features + ['ID']]

In [ ]:
# TODO: test tree behavior in ml_master when given .2 of training set to work with
# TODO: rather than making each tree its own process, divide the number of trees by the number of cores, and create
#       a list of trees that can be processed linearly in its own core.s

num_trees = 4
tries = 1
num_increments = 999
random_seed = 600
random_seed = None
overall_training_ratio = .8
initial_training_ratio = .2
increment_size = 50
cross_val_rnf_incremental(num_trees, df, tries, num_increments, random_seed,cat_features, overall_training_ratio,
                          initial_training_ratio, increment_size)

# Built-in incremental learning vs trees training on larger initial sizes

## Initializing control variables

In [ ]:
n_trees = 64
tree_depth = 100
random_seed = 42
n_max_features = 11
cat_features = ['To', 'From']

## Forests Trained on increasing datasets

In [ ]:
forest_100 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
forest_200 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 200, cat_features)
forest_300 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 300, cat_features)
forest_400 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
forest_500 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 500, cat_features)
incremental_forests = [forest_100, forest_200, forest_300, forest_400, forest_500]

In [ ]:
for forest in incremental_forests:
    forest.fit_parallel()

In [ ]:
# evaluation
for forest in incremental_forests:
    print(evalStats(forest.predict_parallel(df[-100:])[1], df[-100:]), end='\n\n')

##  Incremental Forests

In [ ]:
incremental_forest = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)

In [ ]:
incremental_forest.fit_parallel()

In [ ]:
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

In [ ]:
incremental_forest.update(df[100:200])

In [ ]:
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

In [ ]:
flattened_nodes = []
for tree in incremental_forest.trees:
    print('a')
    flattened_nodes.append(tree.traverse())

In [ ]:
for f in flattened_nodes:
    print(f)
    for node in f:
        if len(node.rows) == 0:
            print("oh no")